In [2]:
# Import standard packages
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Import Data
df = pd.read_csv("/Users/ad/Documents/Exercises/_RgressionModeling/data/signage2015_kloof_seapoint.csv")
df.head()

PermissionError: [Errno 1] Operation not permitted: '/Users/ad/Documents/Exercises/_RgressionModeling/data/signage2015_kloof_seapoint.csv'

The data is multivariate, containing continous and categorical variables, measured in different units. _Width_ and _Height_ are measured in metres, while _Area_ is measured in metres-sqaured. The sign's geodetic vertical and horizontal location relative to the prime meridian at Greenwich and naught at the equator is measured in _latitude_ and _longitude_ decimal degrees, respectively. The _illum_ feature maps _Illuminated_, hence only one of these are needed, noting that the variables represent a qualitative property, i.e., whether a sign is of type illuminated. Other categorical variables are 'Suburb' and 'sign'. Finally, illuminated signs are a sub-category of sign. 

# Descriptive Statitstics

In [ ]:
# Calculate the logDimension and append frame with logValue
def calcLog(dimension): 
    var = 'log'+ dimension
    if var in df.columns: # Check if column exist in df
        return var
    else:
        df[var] = np.log(df[dimension]+1)
        return var
    
def reset_df():
    return df[['sign', 'Illuminated', 'Height', 
               'Width', 'Area', 'longitude', 
               'latitude', 'Suburb', 'illum']]

In [ ]:
fig = plt.figure(figsize=(14, 7), dpi=123, facecolor='#E4E5E9') # Generate the figure
spec = fig.add_gridspec(ncols=7, nrows=3)

# Configure Distribution Plots
g0 = fig.add_subplot(spec[0, :4])
sns.histplot(df[calcLog('Height')], kde=True, color='#657FFB', edgecolor='black') #kde - kernal density estimate
#sns.histplot(x='Height', data=df, kde=True, color='#657FFB', edgecolor='black') 
plt.ylabel('# of Signs', fontsize=11)
plt.xlabel('Log Height', fontsize=11)
plt.xlim(0, 3)
plt.ylim(0, 80)

g1 = fig.add_subplot(spec[1, :4])
sns.histplot(df[calcLog('Width')], kde=True, color='#BAD2EF', edgecolor='black') 
#sns.histplot(x='Width', data=df, kde=True, color='#BAD2EF', edgecolor='black')
plt.ylabel('# of Signs', fontsize=11)
plt.xlabel('Log Width', fontsize=11)
#plt.xlim(0, 20)
plt.ylim(0, 80)

g2 = fig.add_subplot(spec[2, :4])
sns.histplot(df[calcLog('Area')], kde=True, color='#657FFB', edgecolor='black') 
#sns.histplot(x='Area', data=df, kde=True, color='#BAD2EF', edgecolor='black')
plt.ylabel('# of Signs', fontsize=11)
plt.xlabel('Log Area', fontsize=11)
#plt.xlim(0, 20)
plt.ylim(0, 80)

# Configure Box Plots
g3 = fig.add_subplot(spec[:, 4])
sns.boxplot(y=df['Height'], color='#657FFB')
plt.xlabel('Height', labelpad=5, fontsize=10)
plt.ylabel(' ')
plt.ylim(0, 35)

g4 = fig.add_subplot(spec[:, 5])
sns.boxplot(y=df['Width'], color='#BAD2EF')
plt.xlabel('Width', labelpad=5, fontsize=10)
plt.ylabel(' ')
plt.ylim(0, 35)

g5 = fig.add_subplot(spec[:, 6])
sns.boxplot(y=df['Area'], color='#657FFB')
plt.xlabel('Area', labelpad=5, fontsize=10)
plt.ylabel(' ')
plt.ylim(0, 35)

for g in [g0, g1, g2, g3, g4, g5]:
    g.patch.set_alpha(0.0)
    g.spines['top'].set_visible(False)
    g.spines['right'].set_visible(False)
    
fig.suptitle('Distribution Sign Dimensionsin meters and meters squared', fontsize=15, y=1)
plt.show()

df = reset_df()

In [ ]:
fig = plt.figure(figsize=(14, 7), dpi=123, facecolor='#E4E5E9') # Generate the figure
spec = fig.add_gridspec(ncols=6, nrows=3)

g0 = fig.add_subplot(spec[:, 0:2])
sns.boxplot(x=df['Illuminated'], y=df['Height'], color='#BAD2EF')
plt.xlabel('Illuminted (Height)', labelpad=5, fontsize=10)
plt.ylabel('Height')
plt.ylim(0, 35)

g1 = fig.add_subplot(spec[:, 2:4])
sns.boxplot(x=df['Illuminated'], y=df['Width'], color='#BAD2EF')
plt.xlabel('Illuminted (Width)', labelpad=5, fontsize=10)
plt.ylabel('Width')
plt.ylim(0, 35)

g2 = fig.add_subplot(spec[:, 4:6])
sns.boxplot(x=df['Illuminated'], y=df['Area'], color='#BAD2EF')
plt.xlabel('Illuminted (Area)', labelpad=5, fontsize=10)
plt.ylabel('Area')
plt.ylim(0, 35)

for g in [g0, g1, g2, g3]:
    g.patch.set_alpha(0.0)
    g.spines['top'].set_visible(False)
    g.spines['right'].set_visible(False)
    
fig.suptitle('Box Plots for Illuminated Signs Dimensions', fontsize=15, y=1)
plt.show()

df = reset_df()


In [ ]:
# Obtain descriptive statics of continous variables
df[['Height','Width', 'Area']].describe().round(3)

In [ ]:
grouped = df[['Height','Width', 'Area', 'Suburb']].groupby('Suburb')
grouped.describe().round(3)

From the box plots and the frequency distributions, the continuous variables (Height, Width, and Area) are skewed to the right, and several outliers beset the data. However, rather than a measurement error, these are outsized signs. Given 

In [ ]:
# See Korstanje, J. (2022)., p. 267. Machine Learning on Geographical Data Using Python.

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(20, 500))
df[['MarkerSize']] = scaler.fit_transform(df[['Area']])

plt.figure(figsize=(14, 7), dpi=123, facecolor='#E4E5E9')
 
plt.scatter(df[df.Suburb == 'Gardens']['longitude'], 
            df[df.Suburb == 'Gardens']['latitude'], 
            s=df[df.Suburb == 'Gardens']['MarkerSize'], 
            c='none', 
            edgecolors='black')
plt.xlabel('Longitude', labelpad=5, fontsize=10)
plt.ylabel('Latitude')
plt.title('Relative Sign Area - Gardens', fontsize=15, y=1)

In [ ]:
plt.figure(figsize=(14, 7), dpi=123, facecolor='#E4E5E9')

plt.scatter(df[df.Suburb == 'Seapoint']['longitude'], 
            df[df.Suburb == 'Seapoint']['latitude'], 
            s=df[df.Suburb == 'Seapoint']['MarkerSize'], 
            c='none', 
            edgecolors='black')
plt.xlabel('Longitude', labelpad=5, fontsize=10)
plt.ylabel('Latitude')
plt.title('Relative Sign Area - Seapoint', fontsize=15, y=1)

In [ ]:
import contextily

# Generate scatter plot
joint_axes = seaborn.jointplot(
    x='longitude', y='latitude', data=df[df.Suburb == 'Gardens'], s=0.5
)
contextily.add_basemap(
    joint_axes.ax_joint,
    crs="EPSG:4326",
    source=contextily.providers.CartoDB.PositronNoLabels
);

In [ ]:
# Generate scatter plot
import seaborn
seaborn.jointplot(x='longitude', y='latitude', data=df[df.Suburb == 'Gardens'], s=0.5);




seaborn.jointplot(x='longitude', y='latitude', data=df[df.Suburb == 'Seapoint'], s=0.5);
#plt.figure(figsize=(14, 7), dpi=123, facecolor='#E4E5E9')

In [ ]:
df[df.Suburb == 'Gardens']['longitude']

In [ ]:
gardens = df[df.Suburb == 'Gardens']
gardens.head()

# Feature Engineering

## Transform (or Scale) Data

Although some machine learning algorithms (e.g., neural networks and SVM's) are sensitive to outliers and non-normally distributed data (Müller et al., 2017, p. 132), or others (e.g., K-means and K-Nearest Neighbors) that are sensitive to the scale of numeric features, requiring the data to be transformed, in the main, machine learning algorithms learn the distribution embedded within the features themselves (Mueller et al., 2016, p. 226). According to Mueller (2016), transforming the data nonetheless allows for faster algortihm convergence and minimises prediction error.

One way to addresss the above outliers, those outside the interquartile range, is the use of a scaler impervious to outliers. RobustScaler, as the name suggest, use statistics that are robust to outliers (Pedregosa et al., JMLR 12, pp. 2825-2830, 2011; Buitinck et al., 2013).  

In [ ]:
# Scale continous variables, except coordinates (lat, long)
features_to_scale = ['Width', 'Height', 'Area']
dat_to_scale = df[features_to_scale]

# RobustScaler's desensitised to outliers
from sklearn.preprocessing import RobustScaler

robust_scale = RobustScaler()
robust_scale.fit(dat_to_scale)

dat_to_scale = robust_scale.transform(dat_to_scale)

In [ ]:
dat_to_scale = pd.DataFrame(dat_to_scale, columns=features_to_scale)
dat_to_scale.head()

df_scaled = df
df_scaled = df_scaled.assign(Width=dat_to_scale['Width'])
df_scaled = df_scaled.assign(Height=dat_to_scale['Height'])
df_scaled = df_scaled.assign(Area=dat_to_scale['Area'])

In [ ]:
# Instantiate dataframe
#X_train_scale = X_train

# Reassign transformed values to X_train
#X_train_scale = X_train_scale.assign(Width=dat_to_scale[:, 0], 
#                               Height=dat_to_scale[:, 1],
#                               Area=dat_to_scale[:, 2])

# Repeat for test sample
#dat_to_scale = X_test[features_to_scale]
#dat_to_scale = robust_scale.transform(dat_to_scale) 

#X_test_scale = X_test
#X_test_scale = X_test_scale.assign(Width=dat_to_scale[:, 0],
#                                   Height=dat_to_scale[:, 1],
#                                   Area=dat_to_scale[:, 2])

## One Hot Enconding

For classification algorithms, one feature for each qualitative ascription is prefered (Müller et al., 2017). This is achieved by the one-hot-encoding.

In [ ]:
# Ferret out the needed columns
feature_col = ['Suburb', 'sign', 'illum', 'Width', 'Height', 'Area', 'latitude', 'longitude']

# One-Hot-Encoding
df_onehot = pd.get_dummies(df_scaled[feature_col])

# Ferret out new variable names
feature_col_onehot = df_onehot.loc[:, 'Width':'sign_projctng'].columns

df_onehot.head()

## Partition train and test samples

In [ ]:
# Feature (X) and target (y) extraction
X = df_onehot[feature_col_onehot] # Features
y = df_onehot['illum_Yes'] # Target variable

In [ ]:
# Partition train and test samples
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25, random_state=0)

From the descriptive statistics summary and concommitant plots, the data contains several outliers, and are non-normally distributed insofar positvely skewed. Accordingly, the data normality assumption is unmet (i.e., the assumption that the data is obtained from a normally distributed population—Halswanter, 2016, p.97), thus a learning method that makes use of non-parametric inferences would better suit classification. Decision Tree Classifiers are non-parametric supervised machine learners that develop classification rules based on if-then-else desiontry. They are suitable for features measured at different scales, that are nominal, and continous (Müller 2017, p. 83). However, Decisiontree's tend to overfit to the training data.

To counter an overfit, the maximum depth (i.e., the number of consecutive questions, denoted by the number of branching layers) of the tree is controlled. Whereas setting the maximum tree depth is known as pre-pruning, post-pruning encompass dropping or removing uninformative nodes. Below we compare the variation of maximum-depth to model's predictive accuracy. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

maxdepth = []
accuracy = []
crossvalscoreavg = []
modelscore = []

for max_depth in range(1, 15):
    tree = DecisionTreeClassifier(random_state=0, max_depth=max_depth)
    tree.fit(X_train, y_train)
    maxdepth.append(max_depth)
    
    scores = cross_val_score(tree, X_train, y_train, cv=5)
    crossvalscoreavg.append(scores.mean())
    
    y_pred = cross_val_predict(tree, X_test, y_test, cv=5)
    
    accscore = accuracy_score(y_test, y_pred)
    accuracy.append(accscore)
    
    modscore = tree.score(X_test, y_test)
    modelscore.append(modscore)
    
    print("max_depth: {}, accuracy: {:.3f}, avg_crossval: {:.3f}, model_score: {:.3f}".format(max_depth, 
                                                                                          accscore, 
                                                                                          scores.mean(), 
                                                                                          modscore))

plt.plot(maxdepth, crossvalscoreavg, linestyle="dotted", label="cross_val_score")
plt.plot(maxdepth, accuracy, linestyle="dashed", label="accuracy_score")
plt.plot(maxdepth, modelscore, linestyle="dashdot", label="Model Score (Test Set)")

plt.xlabel("Maximum Depth")
#plt.ylabel("Measure")
plt.legend(loc='best')
plt.grid(True)

The Average Cross-Validation score measure the model's generalisation performance in terms of its probability of accurate predicitions on average. From the graph, at depths greater 4,  the average score improves, petering at around 60% accuracy. 

This trend translates to the Model Test Score, i.e., 

Maximum tree depth is compared to accuracy score. We find the highest accuracy (0.575) and cross validation (0.602) is at maximum depth 7.

In [ ]:
from sklearn.model_selection import GridSearchCV
_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14]
max_depth_range = _range
min_samples_leaf_range = _range
param_grid = [{'criterion': ['entropy', 'gini'], 
               'max_depth': max_depth_range},
              {'min_samples_leaf': min_samples_leaf_range}]
grid_searchCV = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, scoring='f1')

In the above case, grid search is ran over two sets of parameters, first with every combination of `criterion` and `max_depth` and second, for `min_samples_leaf` (ref.:Stackoverflow).

REF: https://stackoverflow.com/questions/38709690/scikit-learn-using-gridsearchcv-on-decisiontreeclassifier

In [ ]:
grid_searchCV.fit(X_train, y_train)
print("Test set score: {:.3f}".format(grid_searchCV.score(X_test, y_test)))
print("Best parameters: {}".format(grid_searchCV.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_searchCV.best_score_))
print("Best estimator:\n{}".format(grid_searchCV.best_estimator_))

In [ ]:
param_grid = [{'criterion': ['entropy', 'gini'], 
               'max_depth': max_depth_range},
              {'min_samples_leaf': min_samples_leaf_range}]
# GridSearch with CV Splitter
grid_searchCV = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=strat_shuf_split, scoring='f1')

In [ ]:
grid_searchCV.fit(X_train, y_train)
print("Test set score: {:.3f}".format(grid_searchCV.score(X_test, y_test)))
print("Best parameters: {}".format(grid_searchCV.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_searchCV.best_score_))
print("Best estimator:\n{}".format(grid_searchCV.best_estimator_))

In [ ]:
param_grid = [{'criterion': ['entropy', 'gini'], 
               'max_depth': max_depth_range}]

grid_searchCV = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=strat_shuf_split, scoring='f1')

In [ ]:
grid_searchCV.fit(X_train, y_train)
print("Test set score: {:.3f}".format(grid_searchCV.score(X_test, y_test)))
print("Best parameters: {}".format(grid_searchCV.best_params_))
print("Best cross-validation score: {:.3f}".format(grid_searchCV.best_score_))
print("Best estimator:\n{}".format(grid_searchCV.best_estimator_))

|Grid Search CV Model| Test Score | Best Parmaters| Best Cross Val Score |  
|--------------------|------------|---------------|----------------------|
|CV = 5

In [ ]:
scores = cross_val_score(tree, X, y, cv=11, scoring='f1')
print(scores)
print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)

y_pred = cross_val_predict(tree, X, y, cv=11)
print("Confusion Matrix: \n", format(confusion_matrix(y, y_pred)))
print("\nClassificatin Report:\n", format(classification_report(y, y_pred)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
maxdepth = []
accuracy = []
for max_depth in range(1,14):
    tree = DecisionTreeClassifier(random_state=0, max_depth=max_depth).fit(X_train, y_train)
    predict = tree.predict(X_test)
    maxdepth.append(max_depth)
    accuracy.append(accuracy_score(y_test, predict))
    #print("max_depth {}, accuracy {:.3f}".format(max_depth, accuracy_score(y_test, predict)))

plt.plot(maxdepth, accuracy, linestyle="--")
plt.xlabel("Maximum Depth")
plt.ylabel("Accuracy Score")
plt.grid(True)

In [ ]:
from sklearn.model_selection import learning_curve
train_size, train_scores, test_scores = learning_curve(tree,
                                                      X, y,
                                                      cv=11,
                                                      scoring='f1',
                                                      n_jobs = -1, #Use all the processors
                                                      train_sizes = np.linspace(0.01, 1, 100),
                                                      verbose =1)

In [ ]:
train_mean = np.mean(train_scores, axis=1)
print(train_mean)

In [ ]:
test_mean = np.mean(test_scores, axis=1)
print(test_mean)

In [ ]:
plt.plot(train_size, train_mean, label='Training Score')
plt.plot(train_size, test_mean, label='Cross-Validation Score')

plt.title('Learning Curve')
plt.xlabel('Number of Training Samples')
plt.ylabel('Accuracy')
plt.legend(loc='best')

In [ ]:
tree = DecisionTreeClassifier(random_state=0, max_depth=7).fit(X_train, y_train)
# tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = tree.predict(X_test)

print("Confusion Matrix: \n", format(confusion_matrix(y_test, y_pred)))
print("\nAccuracy Score:", format(accuracy_score(y_test, y_pred)))
print("\nClassificatin Report:\n", format(classification_report(y_test, y_pred)))
print("Training set score: {:.3f}".format(tree.score(X_train, y_train)))
print("\nTest set score: {:.3f}".format(tree.score(X_test, y_test)))

 ## Feature Selection

In [ ]:
correlation_matrix = X.corr()
print('Correlation Matrix: \n{}'.format(correlation_matrix))

In [ ]:
import seaborn as sns
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, mask=np.zeros_like(correlation_matrix, dtype=np.bool),
           cmap=sns.diverging_palette(220, 10, as_cmap=True),
           square=True, ax=ax)

Starting off at the sign dimentions, we find that 
- area has a higher correlation with height (0.813) than width (0.390); 
- compared to the four sign types (i.e., flat-, boundary-, canopy-, and projecting signs) Area has a higher correlation with flat signs (0.137), as does height (0.094) and width (0.314); 
- finally, of the three sign dimensions, area has a higher correlation with illuminated sign (0.060). 

Illuminated signs have a higher correlation with flat signs (0.057). Whereas Gardens have a higher correlation with boundary wall signs(0.153), Seapoint has a higher correlation with flat signs (0.146).
Interestingly, while all signs are negatively correlated, flat sign have the greatest negative correlation with canopy sign (-0.666). A possible reason for this could be where there is no canopy over the pavement, eliminating the choice of suspending a sign from the canopy; flatsigns being the next best option. Here building dimensionality is a case in point for determining sign choice. 

From the above, Features with an absolute correlation greater 0.5 threshold were selected. These

In [ ]:
correlated_feature_matrix = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.6:
            corr_col = correlation_matrix.columns[i]
            correlated_feature_matrix.add(corr_col)

            
print(correlated_feature_matrix)

# Classifiers

## Logistic Regression 

### Without Transformed Data

In [ ]:
# Logistic Regression with one-hot-encoded data
logreg = LogisticRegression().fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Confusion Matrix: \n", format(confusion_matrix(y_test, y_pred)))
print("\nAccuracy Score: {:.3f}".format(accuracy_score(y_test, y_pred)))
print("\nClassificatin Report:\n", format(classification_report(y_test, y_pred)))
print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("\nTest set score: {:.3f}".format(logreg.score(X_test, y_test)))

From the confusion matrix we can conclude that:
- the number of signs correctly classified as illuminated is 36 (i.e., True Positive);
- the number of signs correctly classified as un-illimunated is 59 (i.e., True Negative);
- the number of un-illuminated signs classified as illuminated is 20 (i.e., False Positive);
- and the number of illuminated signs wrongly classified as un-illuminated is 65 (i.e., False Negative)

Accuracy Score = Correct Predictions / Total Predictions. Or the number of successfull predictions:
(True Positives + True Negatives) / (Total in test sample) = (36 + 59) / (0.25 * 720) = 0.528

Recall Score (or true positive rate) = Percentage of illuminated signs identified correctly:
(True Positive) / (True Positve + False Negative) = 36 / (36 + 65) = 0.356

Precision Score = Percentage of signs identified as illuminated that are actually illuminated:
(True Positive) / (True Positive + False Positive) = 36 / (36 + 20) = 0.643

### With Transformed Data

In [ ]:
logreg_scaled = LogisticRegression().fit(X_train_scale, y_train)
y_pred = logreg_scaled.predict(X_test_scale)

print("Confusion Matrix: \n", format(confusion_matrix(y_test, y_pred)))
print("\nAccuracy Score:", format(accuracy_score(y_test, y_pred)))
print("\nClassificatin Report:\n", format(classification_report(y_test, y_pred)))
print("Training set score: {:.3f}".format(logreg_scaled.score(X_train_scale, y_train)))
print("\nTest set score: {:.3f}".format(logreg_scaled.score(X_test_scale, y_test)))

The transformed data delivers slightly better training- and test set scores. However, the recall metric is below desired. As corroborated by the false negative count (i.e., 65), a large proportion of illuminated signs are wrongly classified.  

### Precision-Recall

Assuming that an illuminated sign comes at premium, a lower recall is undesirable say for a cost estimate. One way to adjust the tradeoff between recall and precision is to adjust the threshold at which the model makes the classification decision (Müller et al., 2017, p. 289). 

In [ ]:
from sklearn.metrics import precision_recall_curve

def plot_precision_recall_curve(precision, recall, thresholds, threslabel):
    # Find Threshold closest to zero
    close_zero = np.argmin(np.abs(thresholds))
    plt.plot(precision[close_zero], recall[close_zero], 'o', markersize=10,
        label = threslabel, fillstyle='none', c='k', mew=2)

    plt.plot(precision, recall, label='precision recall curve')
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.legend(loc=2)

precision, recall, thresholds = precision_recall_curve(y_test, logreg_scaled.decision_function(X_test_scale))
plot_precision_recall_curve(precision, recall, thresholds, 'threshold naught')

The `decision_function`'s default threshold is 0—that is the black circle in the *Precision-Recall* graph above—corresponding to 0.64 precision and 0.356 recall. The steep gradient at the beginning, between recall equalling 1 and 0.4, suggest that gain in precision sacrifices plenty recall. On the whole the model has low recall retention as precision increases. Prefered in this case is that recall is kept high as precision increases. That is, a graph that veers toward the upper-right corner, rather than (in the above case) one that recedes there-from (Müller et al., 2017, p. 290).  

Given the graph's naught threshold position (with increasing threshold from left to right), setting the *operating-point* to increase recall would mean reducing the threshold below naught—i.e., shift the threshold point to the left.

In [ ]:
# Reduce the threshold below naught
y_pred_lower_threshold = logreg_scaled.decision_function(X_test_scale) > -0.8
print(classification_report(y_test, y_pred_lower_threshold))

print("\nConfusion Matrix: \n", format(confusion_matrix(y_test, y_pred_lower_threshold)))

from sklearn.metrics import average_precision_score
print("\nAverage precision {:.3f}".format(average_precision_score(y_test, y_pred_lower_threshold)))

plot_precision_recall_curve(precision, recall, y_pred_lower_threshold, 'adjusted threshold')

Although adjusting the threshold yields significant recall improvement (from 0.36 to 0.96 in the above graph), slightly decreasing precision (from 0.65 to 0.58), the number of false positives shot up (from 19 to 71). Further, the average precision—i.e., the area under the precision-recall curve—totals 0.577. This  is slightly off the midrange mark (0.5) for average precision (Müller et al., 2017, p. 292). For this reason, it's concluded that the model has average precsion.   

Precision and recall are important meausures for determining model success, not least for 'class-imbalenced problems' (Chollet, 2018, p. 112). For 'balanced classification problems' the *receiver operating characteristc* (ROC) and *area under the curve* (AUC) metric are recommended. Before calculating these, class imbalance occur when 'one class is more frequent than the other', also called an imbalanced data set (Müller et al., 2017, p. 277). In this example the number of illuminated signs relatively equal the un-illuminated signs (352 illuminated out of 720 signs), suggesting that the ROC and AUC metric are better measures of model success (Chollet, 2018, p. 112).   

In [ ]:
df_onehot['illum_Yes'].value_counts()

The ROC curve plots the *True Positive Rate* (TPR) or Recall on the y-axis against the *False Positive Rate* (FPR) on the x-axis at various thresholds. A suitable ROC curve yields high Recall and low FPR—a curve that traverses upwards from the origin toward the top left corner, petering out at the top right corner (Müller et al., 2017, p. 293). The AUC is the area under the ROC curve,  thus:   

In [ ]:
from sklearn.metrics import roc_auc_score
logreg_auc = roc_auc_score(y_test, logreg_scaled.decision_function(X_test_scale))

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, logreg_scaled.decision_function(X_test_scale))

def plot_custom_roc(fpr, tpr, thresholds):
    plt.plot(fpr, tpr, label="ROC Curve (AUC = %0.3f)" % logreg_auc)
    plt.xlabel("FPR (False Positive Rate)")
    plt.ylabel("TPR (True Positive Rate)")
    
    # Find threshold closest to zero
    close_zero = np.argmin(np.abs(thresholds))
    plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize=10,
         label = 'naught threshold @ (fpr = %0.3f, ' % fpr[close_zero] + 'tpr = %0.3f)' % tpr[close_zero], 
         fillstyle='none', c='k', mew=2)
    
    plt.plot([0, 1], linestyle=":")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.grid(True)
    plt.legend(loc=4)

plot_custom_roc(fpr, tpr, thresholds)

In [ ]:
dud = pd.DataFrame()
type(dud)

box = []
type(box)

The naught (or default) threshold positions at 0.266 FPR and 0.356 TPR. The AUC (0.566) suggest that there's 56.6% 'probability that a randomly picked point of the positive class will have a higher score according to the classifier than a randomly picked point from the negative class' (Müller et al., 2017, p. 295). Hence the classifier  predicts slighty better than a random-guess predictor with AUC = 0.5 (i.e., the area under the diagonal line).

### Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score

logreg_cross_val = LogisticRegression(max_iter=200).fit(X_train_scale, y_train)
scores = cross_val_score(logreg_cross_val, X_train_scale, 
                         y_train, cv=10, scoring='f1')
print(
    'Cross-validation scores (10 splits): \n {}'.format(scores)
)

print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)

#print('Cross Val Score: {}'.format(logreg_cross_val.score(X_test_scale, y_test)))
#print('Average cross-validation score: {:.3f}'.format(scores.mean()))

#### k-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10)
scores = cross_val_score(logreg_cross_val, X_train_scale, 
                         y_train, cv=kfold, scoring='f1')
print('Cross-validation scores (10 splits): \n {}'.format(scores))
print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)
# print('\nAverage cross-validation score: {:.3f}'.format(scores.mean()))

#### Repeated k-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import RepeatedKFold
repeated_kfold = RepeatedKFold(n_splits=10, n_repeats=2)
scores = cross_val_score(logreg_cross_val, X_train_scale, 
                         y_train, cv=repeated_kfold, scoring='f1')
print('Cross-validation scores (10 splits): \n {}'.format(scores))
print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)
# print('\nAverage cross-validation score: {:.3f}'.format(scores.mean()))

#### Shuffle k-Fold Cross-Validation

In [ ]:
kfold_shuffle = KFold(n_splits=10, shuffle=True, random_state=0)
scores = cross_val_score(logreg_cross_val, X_train_scale, 
                         y_train, cv=kfold_shuffle, scoring='f1')
print('\nCross-validation scores (10 splits, shuffle=True, random_state=0): \n {}'.format(scores))
print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)

print('\nAverage cross-validation score: {:.3f}'.format(scores.mean()))

In [ ]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(n_splits=10, random_state=0)
scores = cross_val_score(logreg_cross_val, X_train_scale, 
                         y_train, cv=shuffle_split, scoring='f1')
print('\nCross-validation scores (10 splits, shuffle=True, random_state=0): \n {}'.format(scores))
print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)

print('\nAverage cross-validation score: {:.3f}'.format(scores.mean()))

#### Stratified k-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_val_score(logreg_cross_val, X_train_scale, 
                         y_train, cv=skf, scoring='f1')
print('\nStratifiedKFold Cross-validation scores (10 splits, shuffle=True, random_state=0): \n {}'.format(scores))
print(
    "\n%0.3f accuracy with a standard deviation of %0.3f" 
    % (scores.mean(), scores.std())
)

print('\nAverage cross-validation score: {:.3f}'.format(scores.mean()))

## Decision Trees Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=0, max_depth=7).fit(X_train, y_train)
# tree = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
y_pred = tree.predict(X_test)

print("Confusion Matrix: \n", format(confusion_matrix(y_test, y_pred)))
print("\nAccuracy Score:", format(accuracy_score(y_test, y_pred)))
print("\nClassificatin Report:\n", format(classification_report(y_test, y_pred)))
print("Training set score: {:.3f}".format(tree.score(X_train, y_train)))
print("\nTest set score: {:.3f}".format(tree.score(X_test, y_test)))

In [ ]:
from sklearn.metrics import r2_score
maxdepth = []
accuracy = []
for max_depth in range(1,14):
    tree = DecisionTreeClassifier(random_state=0, max_depth=max_depth).fit(X_train_scale, y_train)
    predict = tree.predict(X_test_scale)
    maxdepth.append(max_depth)
    accuracy.append(accuracy_score(y_test, predict))
    #print("max_depth {}, accuracy {:.3f}".format(max_depth, accuracy_score(y_test, predict)))

plt.plot(maxdepth, accuracy, linestyle="--")
plt.xlabel("Maximum Depth")
plt.ylabel("Accuracy Score")
plt.grid(True)

fpr, tpr, thresholds = roc_curve(y_test, tree.decision_function(X_test_scale))
plot_custom_roc(fpr, tpr, thresholds)

In [ ]:
from sklearn import tree

tree_model = DecisionTreeClassifier(random_state=0, max_depth=3).fit(X_train_scale, y_train)

plt.figure(figsize=(50,20))
tree.plot_tree(tree_model, feature_names=X_train_scale.columns)
plt.show()

### Feature Importance

In [ ]:
plt.plot(logreg.coef_.T, 'o', label="C=1")
plt.plot(logreg.fit(.coef_.T, '^', label="C=100")
plt.xticks(range(X.shape[1]), feature_cols_onehot, rotation=90)
plt.hlines(0, 0, X.shape[1])
plt.ylim(-5, 5)
plt.xlabel("Coefficient index")
plt.ylabel("Coefficient magnitude")
plt.title("Coefieicients learned by logistic regression with L2 penalty for different C values, where C is the strength regularisation")
plt.legend()

In [ ]:
for var, coef in zip(feature_cols, 
                    logreg.coef_[0]):
    print('%7s : %7.3f' %(var, coef))

In [ ]:
def plot_feature_importance(model):
    n_features = X.shape[1]
    plt.barh(range(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), feature_cols)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    
plot_feature_importance(tree)

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Build 5 trees
forest = RandomForestClassifier(n_estimators=10, 
                                max_features="sqrt", 
                                max_depth=None, 
                                min_samples_split=2,
                                bootstrap=True)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

print("Confusion Matrix: \n", format(confusion_matrix(y_test, y_pred)))
print("\nAccuracy Score:", format(accuracy_score(y_test, y_pred)))
print("\nClassificatin Report:\n", format(classification_report(y_test, y_pred)))
print("Training set score: {:.3f}".format(forest.score(X_train, y_train)))
print("\nTest set score: {:.3f}".format(forest.score(X_test, y_test)))

plot_feature_importance(forest)

In [ ]:
import mglearn
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
for i, (ax, tree) in enumerate(zip(axes.ravel(), forest.estimators_)):
    ax.set_title("Tree {}".format(i))
    mglearn.plots.plot_tree_partition(X_train, y_train, tree, ax=ax)

mglearn.plots.plot_2d_seperator(forest, X_train, fill=True, ax=axes[-1, -1],
                               alpha=.4)
axes[-1, -1].set_title("Random Forest")
mglearn.discrete_scatter(X_train[:, 0], X_train[:, 1], y_train)